In [35]:
import os
import numpy as np
from scipy import *
from scipy.sparse import *
from scipy.spatial import distance
from scipy.stats import entropy
import pandas as pd
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
def sparseload(file):
    f = open(file)
    h = f.readline()
    col = []
    data = []
    chrom = []
    for l in f:
        l = l.strip()
        chrom.append(l.split('\t')[0])
        col.append(int(l.split('\t')[1]))
        data.append(int(float(l.split('\t')[2])))
    col = np.array(col)
    data = np.array(data)
    chrom = np.array(chrom)
    f.close()
    return chrom,col,data

In [37]:
'''Experiment is one of: xz_10_15, xz_13, xz_14, xz_13_14 '''
def structure_genome(resolution):
    f = open('/home/garner1/Work/pipelines/data/relevant-chr-size_list.txt')
    base = 0
    positionlist = []
    chromlist = []
    border = []
    for l in f.readlines():
        l = l.strip()
        chrom = l.split('\t')[0]
        size = l.split('\t')[1]
        end = base + int(size)/resolution
        border.append(end)
        array = arange(base, end)
        positionlist.extend(array)
        chromlist.extend([chrom]*len(array))
        base = base + int(size)/resolution
    f.close()
    return positionlist,chromlist,border
# positionlist,chromlist,border = structure_genome(10000000)

In [38]:
def load_data(experiment,resolution):
    regions = os.listdir('/home/garner1/Work/dataset/restseq/data/'+str(experiment)+'/'+str(resolution))
    length = 0
    row_list = []
    index0_list = []
    index1_list = []
    index2_list = []
    index3_list = []
    for region in sorted(regions):
        filename = '/home/garner1/Work/dataset/restseq/data/'+str(experiment)+'/'+str(resolution)+'/'+region
        if experiment=='xz_13' or experiment=='xz_14':
            index0 = region[0]
            index1 = region[2]
            index0_list.append(index0)
            index1_list.append(index1)
        if experiment=='xz_10_15':
            index0 = region[0]
            index1 = region[2]
            if len(region[3:len(region)-4])==3:
                index2 = region[3:len(region)-4][0]
                index3 = region[3:len(region)-4][2]
            if len(region[3:len(region)-4])==2:
                index2 = 'A'
                index3 = region[3:len(region)-4][1]
            index0_list.append(index0)
            index1_list.append(index1)
            index2_list.append(index2)
            index3_list.append(index3)
        if experiment=='xz_13_14':
            index0 = region[:4]
            index1 = region[5]
            index2 = region[7]
            index0_list.append(index0)
            index1_list.append(index1)
            index2_list.append(index2)
        chrom, col, data = sparseload(filename)
        row = np.zeros(len(col))
        vec_sparse = csc_matrix((data,(row,col-1)), shape=(1,len(positionlist)))
        row_list.append(vec_sparse.toarray())
    X = np.vstack( row_list )
    return X,index0_list,index1_list,index2_list,index3_list
# X,index0_list,index1_list,index2_list,index3_list = load_data('xz_13',10000000)

In [39]:
def load_RCdf(X,experiment,chromlist,positionlist,groupby,RC_threshold):
    arrays = [chromlist, positionlist]
    columns = pd.MultiIndex.from_arrays(arrays, names=['chromosome', 'bin'])
    if experiment=='xz_10_15':
        row = pd.MultiIndex.from_arrays([index0_list,index1_list,index2_list,index3_list],\
                                    names=['patient','type','set','fragment'])
    if experiment=='xz_13_14':
        row = pd.MultiIndex.from_arrays([index0_list,index1_list,index2_list],\
                                    names=['experiment','row','column'])
    if experiment=='xz_13' or experiment=='xz_14':
        row = pd.MultiIndex.from_arrays([index0_list,index1_list],\
                                    names=['row','column'])
    X_df = pd.DataFrame(data=X,index=row,columns=columns)
    '''
    Remove rows with 0 read counts
    '''
    X_df = X_df.loc[~(X_df==0).all(axis=1)]

    if groupby=='10-15-reduced':
        X_red_df = X_df.groupby(level=['patient','type','set']).sum()
    if groupby=='13-14-reduced':
        X_red_df = X_df.groupby(level=['row','column']).sum()
    if groupby=='none':
        X_red_df = X_df
    '''
    Set zero columns
    '''
    mask = X==0
    row_sum = (~mask).sum(axis=0)
    mask = row_sum==0
    zero_col = np.arange(X.shape[1])[mask]
    '''
    Remove columns with 0 read counts
    '''
    X_red_df = X_red_df[X_red_df.columns[(X_red_df != 0).any()]]
    '''
    Remove rows with read counts less than threshold
    '''
    X_red_df = X_red_df.loc[~(X_red_df.sum(axis=1)<RC_threshold)]
    '''
    Define the data matrix
    '''
    X = X_red_df.as_matrix()
    return X_df,X_red_df,X,zero_col
# X_df,X_red_df,X,zero_col = load_RCdf(X,'xz_13',chromlist,positionlist,'none',1000)

In [40]:
def impute(X,zero_col):
    X_incomplete = X.copy()
    X_incomplete = X_incomplete*1.0
    X_incomplete[X_incomplete==0] = np.nan
    softImpute = SoftImpute(max_iters=100,n_power_iterations=3,min_value=0,verbose=False)
    # # simultaneously normalizes the rows and columns of your observed data,
    # # sometimes useful for low-rank imputation methods
    biscaler = BiScaler()
    # # rescale both rows and columns to have zero mean and unit variance
    X_incomplete_normalized = biscaler.fit_transform(X_incomplete)
    X_filled_softimpute_normalized = softImpute.complete(X_incomplete_normalized)
    X_filled_softimpute = biscaler.inverse_transform(X_filled_softimpute_normalized)
    X_filled_softimpute = softImpute.complete(X_incomplete)
    '''
    Re-insert the zeros cols
    '''
    zero_col = zero_col[:-2]
    X_filled_softimpute = np.insert(X_filled_softimpute,zero_col,0,axis=1) 
    return X_filled_softimpute
# X_filled_softimpute = impute(X,zero_col)

[BiScaler] Initial log residual value = 14.039364
[BiScaler] Iter 1: log residual = 4.014279, log improvement ratio=10.025085
[BiScaler] Iter 2: log residual = 3.281676, log improvement ratio=0.732603
[BiScaler] Iter 3: log residual = 2.003150, log improvement ratio=1.278526
[BiScaler] Iter 4: log residual = 0.399320, log improvement ratio=1.603830
[BiScaler] Iter 5: log residual = -1.218663, log improvement ratio=1.617983
[BiScaler] Iter 6: log residual = -2.786118, log improvement ratio=1.567455
[BiScaler] Iter 7: log residual = -4.291910, log improvement ratio=1.505792
[BiScaler] Iter 8: log residual = -5.748049, log improvement ratio=1.456139
[BiScaler] Iter 9: log residual = -7.169094, log improvement ratio=1.421045
[BiScaler] Iter 10: log residual = -8.566925, log improvement ratio=1.397831
[BiScaler] Iter 11: log residual = -9.949578, log improvement ratio=1.382653
[BiScaler] Iter 12: log residual = -11.322263, log improvement ratio=1.372685
[BiScaler] Iter 13: log residual = -1

In [41]:
'''
Normalize rows to 1
'''
def normalize(X_filled_softimpute):
    col_sum = X_filled_softimpute.sum(axis=1)
    X_filled_normalized = np.zeros(X_filled_softimpute.shape)
    for ind in range(X_filled_softimpute.shape[0]):
        X_filled_normalized[ind,:] = X_filled_softimpute[ind,:]*1.0/col_sum[ind]
    X_filled_normalized_df = pd.DataFrame(data=X_filled_normalized,index=X_red_df.index)
    X_filled_df = pd.DataFrame(data=X_filled_softimpute,index=X_red_df.index)
    return X_filled_normalized_df,X_filled_df
# X_filled_normalized_df,X_filled_df = normalize(X_filled_softimpute)

In [42]:
def shannonDF(X_filled_df):
    index = ['A','B','C','D','E','F','G','H','I','J']
    columns = ['1','2','3','4','5','6','7']
    shannon_df_xz1314 = pd.DataFrame(0,index=index, columns=columns)
    shannon_df_xz13 = pd.DataFrame(0,index=index, columns=columns)
    shannon_df_xz14 = pd.DataFrame(0,index=index, columns=columns)
    shannon = []
    df = X_filled_df
    for i in df.index:
        shannon.append( entropy(df.ix[i]) )
    for i in range(len(df.index.values)):
        if df.index.values[i][0]=='xz13':
            shannon_df_xz13.ix[df.index.values[i][1],df.index.values[i][2]] = shannon[i]
        if df.index.values[i][0]=='xz14':
            shannon_df_xz14.ix[df.index.values[i][1],df.index.values[i][2]] = shannon[i]
        if df.index.values[i][0]!='xz13' and df.index.values[i][0]!='xz14':
            shannon_df_xz1314.ix[df.index.values[i][0],df.index.values[i][1]] = shannon[i]
    return shannon_df_xz1314,shannon_df_xz13,shannon_df_xz14
# shannon_df_xz1314,shannon_df_xz13,shannon_df_xz14 = shannonDF(X_filled_df)

In [43]:
def readcountDF(X_filled_df):
    index = ['A','B','C','D','E','F','G','H','I','J']
    columns = ['1','2','3','4','5','6','7']
    totalRC_df_xz1314 = pd.DataFrame(0,index=index, columns=columns)
    totalRC_df_xz13 = pd.DataFrame(0,index=index, columns=columns)
    totalRC_df_xz14 = pd.DataFrame(0,index=index, columns=columns)
    totalRC = []
    df = X_filled_df
    for i in df.index:
        totalRC.append( df.ix[i].sum(axis=0) )
    for i in range(len(df.index.values)):
        if df.index.values[i][0]=='xz13':
            totalRC_df_xz13.ix[df.index.values[i][1],df.index.values[i][2]] = totalRC[i]
        if df.index.values[i][0]=='xz14':
            totalRC_df_xz14.ix[df.index.values[i][1],df.index.values[i][2]] = totalRC[i]
        if df.index.values[i][0]!='xz13' and df.index.values[i][0]!='xz14':
            totalRC_df_xz1314.ix[df.index.values[i][0],df.index.values[i][1]] = totalRC[i]
    return totalRC_df_xz1314,totalRC_df_xz13,totalRC_df_xz14
# totalRC_df_xz1314,totalRC_df_xz13,totalRC_df_xz14 = readcountDF(X_filled_df)

In [45]:
'''
Distance between regions
'''
def NNdistDF_1314(X_filled_df,shannon_df_xz1314):
    index = ['A','a-b','B','b-c','C','c-d','D','d-e','E','e-f','F','f-g','G','g-h','H','h-i','I','i-j','J']
    columns = ['1','1-2','2','2-3','3','3-4','4','4-5','5','5-6','6','6-7','7']
    delta_df_xz1314 = pd.DataFrame(0,index=index, columns=columns)
    df = X_filled_df
    rr = 0
    for row in range(len(shannon_df_xz1314.ix[:,0])-1):
        cc = 1
        for col in range(len(shannon_df_xz1314.ix[0,:])-1):
            if totalRC_df_xz1314.ix[row,col]!=0 and totalRC_df_xz1314.ix[row,col+1]!=0:
                ind1 = str(shannon_df_xz1314.index.values[row])
                ind2 = str(shannon_df_xz1314.columns.values[col])
                ind3 = str(shannon_df_xz1314.columns.values[col+1])
                delta_df_xz1314.ix[index[rr],columns[cc]] = distance.cosine(df.ix[ind1,ind2],df.ix[ind1,ind3])
            cc = cc+2
        rr = rr+2
    rr = 1
    for row in range(len(shannon_df_xz1314.ix[:,0])-1):
        cc = 0
        for col in range(len(shannon_df_xz1314.ix[0,:])-1):
            if totalRC_df_xz1314.ix[row,col]!=0 and totalRC_df_xz1314.ix[row+1,col]!=0:
                ind1 = str(shannon_df_xz1314.index.values[row])
                ind2 = str(shannon_df_xz1314.columns.values[col])
                ind3 = str(shannon_df_xz1314.index.values[row+1])
                delta_df_xz1314.ix[index[rr],columns[cc]] = distance.cosine(df.ix[ind1,ind2],df.ix[ind3,ind2])
            cc = cc+2
        rr = rr+2
    return delta_df_xz1314
# delta_df_xz1314 = NNdistDF_1314(X_filled_df,shannon_df_xz1314)

In [46]:
'''
Fluctuations with respect to the average
'''
def fluctuationDF_1314(X_filled_df,X_filled_softimpute):
    mean = X_filled_softimpute.sum(axis=0)
    mean = mean*1.0/mean.sum()
    index = ['A','B','C','D','E','F','G','H','I','J']
    columns = ['1','2','3','4','5','6','7']
    fluctuation_df_xz1314 = pd.DataFrame(0,index=index, columns=columns)
    fluctuation = []
    df = X_filled_df
    for i in df.index:
        fluctuation.append( distance.cosine(df.ix[i],mean) )
    for i in range(len(df.index.values)):
        fluctuation_df_xz1314.ix[df.index.values[i][0],df.index.values[i][1]] = fluctuation[i]
#     fluctuation_df_xz1314 = fluctuation_df_xz1314.replace([np.inf], -1.0)
#     fluctuation_df_xz1314 = fluctuation_df_xz1314.replace( -1.0, 2.0*fluctuation_df_xz1314.max().max() )
    return fluctuation_df_xz1314
# fluctuation_df_xz1314 = fluctuationDF_1314(X_filled_df,X_filled_softimpute)

In [47]:
'''
Distance between nearby regions
'''
def NNdistDF(X_filled_df):
    index = ['A','a-b','B','b-c','C','c-d','D','d-e','E','e-f','F','f-g','G','g-h','H','h-i','I','i-j','J']
    columns = ['1','1-2','2','2-3','3','3-4','4','4-5','5','5-6','6','6-7','7']
    delta_df_xz13 = pd.DataFrame(0,index=index, columns=columns)
    delta_df_xz14 = pd.DataFrame(0,index=index, columns=columns)
    rr = 0
    df = X_filled_df
    for row in range(len(shannon_df_xz13.ix[:,0])-1):
        cc = 1
        for col in range(len(shannon_df_xz13.ix[0,:])-1):
            if totalRC_df_xz13.ix[row,col]!=0 and totalRC_df_xz13.ix[row,col+1]!=0:
                ind0 = 'xz13'
                ind1 = str(shannon_df_xz13.index.values[row])
                ind2 = str(shannon_df_xz13.columns.values[col])
                ind3 = str(shannon_df_xz13.columns.values[col+1])
                delta_df_xz13.ix[index[rr],columns[cc]] = distance.cosine(df.ix[ind0,ind1,ind2],df.ix[ind0,ind1,ind3])
            cc = cc+2
        rr = rr+2
    rr = 1
    for row in range(len(shannon_df_xz13.ix[:,0])-1):
        cc = 0
        for col in range(len(shannon_df_xz13.ix[0,:])-1):
            if totalRC_df_xz13.ix[row,col]!=0 and totalRC_df_xz13.ix[row+1,col]!=0:
                ind0 = 'xz13'
                ind1 = str(shannon_df_xz13.index.values[row])
                ind2 = str(shannon_df_xz13.columns.values[col])
                delta_df_xz13.ix[index[rr],columns[cc]] = distance.cosine(df.ix[ind0,ind1,ind2],df.ix[ind0,ind3,ind2])
            cc = cc+2
        rr = rr+2
    rr = 0
    for row in range(len(shannon_df_xz14.ix[:,0])-1):
        cc = 1
        for col in range(len(shannon_df_xz14.ix[0,:])-1):
            if totalRC_df_xz14.ix[row,col]!=0 and totalRC_df_xz14.ix[row,col+1]!=0:
                ind0 = 'xz14'
                ind1 = str(shannon_df_xz14.index.values[row])
                ind2 = str(shannon_df_xz14.columns.values[col])
                ind3 = str(shannon_df_xz14.columns.values[col+1])
                delta_df_xz14.ix[index[rr],columns[cc]] = distance.cosine(df.ix[ind0,ind1,ind2],df.ix[ind0,ind1,ind3])
            cc = cc+2
        rr = rr+2
    rr = 1
    for row in range(len(shannon_df_xz14.ix[:,0])-1):
        cc = 0
        for col in range(len(shannon_df_xz14.ix[0,:])-1):
            if totalRC_df_xz14.ix[row,col]!=0 and totalRC_df_xz14.ix[row+1,col]!=0:
                ind0 = 'xz14'
                ind1 = str(shannon_df_xz14.index.values[row])
                ind2 = str(shannon_df_xz14.columns.values[col])
                ind3 = str(shannon_df_xz14.index.values[row+1])
                delta_df_xz14.ix[index[rr],columns[cc]] = distance.cosine(df.ix[ind0,ind1,ind2],df.ix[ind0,ind3,ind2])
            cc = cc+2
        rr = rr+2
    return delta_df_xz13,delta_df_xz14
# delta_df_xz13,delta_df_xz14 = NNdistDF(X_filled_df)

In [48]:
'''
Fluctuations with respect to the average, measured with KL
'''
def fluctuationDF(X_filled_df,X_filled_softimpute):
    mean = X_filled_softimpute.sum(axis=0)
    mean = mean*1.0/mean.sum()
    index = ['A','B','C','D','E','F','G','H','I','J']
    columns = ['1','2','3','4','5','6','7']
    fluctuation_df_xz13 = pd.DataFrame(0,index=index, columns=columns)
    fluctuation_df_xz14 = pd.DataFrame(0,index=index, columns=columns)
    fluctuation = []
    df = X_filled_df
    for i in df.index:
        fluctuation.append( distance.cosine(df.ix[i],mean) )
    for i in range(len(df.index.values)):
        if df.index.values[i][0]=='xz13':
            fluctuation_df_xz13.ix[df.index.values[i][1],df.index.values[i][2]] = fluctuation[i]
        if df.index.values[i][0]=='xz14':
            fluctuation_df_xz14.ix[df.index.values[i][1],df.index.values[i][2]] = fluctuation[i]
    return fluctuation_df_xz13,fluctuation_df_xz14
# fluctuation_df_xz13,fluctuation_df_xz14 = fluctuationDF(X_filled_df,X_filled_softimpute)

In [55]:
'''
Fluctuations between co-localized regions, measured with KL
'''
def local_fluctuationsDF(X_filled_df):
    index = ['A','B','C','D','E','F','G','H','I','J']
    columns = ['1','2','3','4','5','6','7']
    local_fluctuation_df = pd.DataFrame(0,index=index, columns=columns)
    fluctuation = []
    df = X_filled_df
    if len(df.index.values[0])>2:
        for i in range(len(df.index.values)):
            if ('xz13',df.index.values[i][1],df.index.values[i][2]) in df.index and ('xz14',df.index.values[i][1],df.index.values[i][2]) in df.index:
                pr1 = df.ix['xz13',df.index.values[i][1],df.index.values[i][2]]
                pr2 = df.ix['xz14',df.index.values[i][1],df.index.values[i][2]]
                local_fluctuation_df.ix[ df.index.values[i][1],df.index.values[i][2] ] = distance.cosine(pr1,pr2)
    return local_fluctuation_df
# local_fluctuation_df = local_fluctuations(X_filled_df)